In [188]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [ ]:
##Размещение баннеров


In [ ]:
Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

In [214]:
df = pd.read_csv('checkins.dat', sep='|').drop(index=0)

/Users/OUT-Kovyazin-AA/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [215]:
df.head(10)

,id,user_id,venue_id,latitude,longitude,created_at
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
5,984249,2146840.0,5222.0,,,2012-04-21 17:42:58
6,984268,2146843.0,5222.0,,,2012-04-21 17:42:38
7,984281,2146846.0,5222.0,,,2012-04-21 17:39:40
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
9,6651,1338710.0,219703.0,,,2011-12-08 23:11:23
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46


In [216]:
df.columns = [col.strip() for col in df.columns]
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [ ]:
С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry

In [228]:
df = df[['latitude', 'longitude']]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)

df = df.dropna()

In [229]:
df = df.applymap(float)
df = df.round(decimals=6)
data_sample = df.sample(100000)
data_sample.head()

,latitude,longitude
976284,33.422268,-111.822640
997878,38.895112,-77.036366
902050,33.494170,-111.926052
561218,34.159172,-118.501195
543418,33.494170,-111.926052


In [230]:
np.isfinite(df.all())

latitude     True
longitude    True
dtype: bool

In [324]:
%%time
cluster = MeanShift(bandwidth = 0.1)
cluster.fit(data_sample)

CPU times: user 2min 16s, sys: 1.54 s, total: 2min 17s
Wall time: 2min 18s


MeanShift(bandwidth=0.1)

In [368]:
labels = cluster.labels_
cluster_centers = cluster.cluster_centers_

In [369]:
# data_sample['cluster'] = cluster.predict(data_sample)


In [370]:
# cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
# cluster_size.columns = ['clust_size']
# cluster_size.head()

In [371]:
# cluster_centers_df = pd.DataFrame(cluster.cluster_centers_)
# cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [388]:
# labels = np.unique(labels)
clean_center = []
for l in np.unique(labels):
    if list(labels).count(l) > 15:
        clean_center.append(cluster_centers[l])
len(clean_center)
# # labels[:10]

605

In [392]:
filter_c = []
for arr in clean_center:
    filter_c.append([arr[0], arr[1]])
filter_c[:3]

[[40.71734725056607, -73.9894437878442],
 [41.87816290818484, -87.62982539206956],
 [33.44842949516833, -112.07407896530222]]

In [ ]:
Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)
25.867736, -80.324116 (Miami)
51.503016, -0.075479 (London)
52.378894, 4.885084 (Amsterdam)
39.366487, 117.036146 (Beijing)
-33.868457, 151.205134 (Sydney)
Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [375]:
def distance(l1, l2):
    return (l1[0] - l2[0])**2 + (l1[1] - l2[1])**2

In [394]:
offices = [[33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084], 
           [39.366487, 117.036146], [-33.868457, 151.205134]]
min_dists = []
for of in offices:
    dist = [(distance(of, x), x) for x in filter_c]
#     print(dist[:3])
    min_dists.append(min(dist))
    
print(min(min_dists))

(2.6315938360748748e-06, [-33.86776034920633, 151.20659901587285])
